In [1]:
from datetime import datetime
import hashlib

class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_contents = str(self.index) + str(self.timestamp) + str(self.data) + str(self.previous_hash)
        return hashlib.sha256(block_contents.encode()).hexdigest()

def make_genesis_block():
    """Make the first block in a block-chain."""
    block = Block(index=0,
                  timestamp=datetime.now(),
                  data="Genesis Block",
                  previous_hash="0")
    return block

genesis_block = make_genesis_block()

print("Genesis Block Details:")
print("Index:", genesis_block.index)
print("Timestamp:", genesis_block.timestamp)
print("Data:", genesis_block.data)
print("Previous Hash:", genesis_block.previous_hash)
print("Hash:", genesis_block.hash)


Genesis Block Details:
Index: 0
Timestamp: 2023-10-28 00:34:08.596560
Data: Genesis Block
Previous Hash: 0
Hash: 91c393b9104e550c520725975aaed84a1a85f02cfb8dc87ccbbce05022b5bb42


For the second part of this assignment, you need to write a function to generate a public, private
key pair for each requesting node. You need to store your public keys in a way that it is
accessible to everyone. For this purpose, cryptographic libraries are available.

In [2]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

def generate_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()

    public_key_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open("public_key.pem", "wb") as file:
        file.write(public_key_pem)

    return private_key, public_key

private_key, public_key = generate_key_pair()

print("Public key:")
print(public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).decode())



Public key:
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEArmvwABr1vsJ+B82ksFiW
4tovRlztLOzwiJ5fB7qy4LPxJtMZr7greWt6ESW+20vTb1MO+uI27Ah2zUYkkbjp
toyJGAPhXWzg8qxW4DBWV4/n0x3w2Veuvzx1LAc1IpEO9e+iKLXhj/Lu+cNLA6LZ
Fx9DT7GwfE4H/h5diB3Mw7PCimdiR+kFSB5SiamxwVVN70+z1//vRU4BEoNJbQvO
7tl6+MlGu8yRJQ2/WNovvEiQiWTAQuQ1lnmJU6/jKXIEdH9jEdyhjSjXReJ7IqpV
4p74n4tDNt8jFiOOvrqUy7ZA2HB0u02B4fVNbGCoLZX/7WSPssG6WiIBxJZDUVEZ
YwIDAQAB
-----END PUBLIC KEY-----



For the third part of this assignment, you need to write a function to generate the digital
signature using ECDSA for your transaction data. For this purpose, cryptographic libraries are
available.

In [3]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes

def generate_ECDSA_signature(private_key, data):
    signature = private_key.sign(
        data,
        ec.ECDSA(hashes.SHA256())
    )
    return signature

private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())

data = b"Transaction data to be signed"

signature = generate_ECDSA_signature(private_key, data)

print("Generated ECDSA Signature:")
print(signature.hex())


Generated ECDSA Signature:
3045022036e7347b2a9bc2aefa4a548d2ce495a950b65d9c770770af9cfd039a1b0f13b1022100ad6e13768a2ca0f98f0a492ace58f43ece46fbda24c72cd1ad54d831078bb60d


For the fourth part of this assignment, you need to write a function to build a new block based
on the previous block. You may use the following function header for your implementation:
def next_block(pre_block, data='')
Also for mining purpose, you can choose a difficulty which mines a block in atleast 5 mins,
not less than that.

In [4]:
from datetime import datetime
import hashlib
import time

class Block:
    def __init__(self, index, timestamp, data, previous_hash, nonce=0):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_contents = str(self.index) + str(self.timestamp) + str(self.data) + str(self.previous_hash) + str(self.nonce)
        return hashlib.sha256(block_contents.encode()).hexdigest()

def next_block(prev_block, data=''):
    index = prev_block.index + 1
    timestamp = datetime.now()
    previous_hash = prev_block.hash
    nonce = 0
    new_block = Block(index, timestamp, data, previous_hash, nonce)

    difficulty = 4  # Adjust the difficulty as required
    while not new_block.hash.startswith('0' * difficulty):
        nonce += 1
        new_block.nonce = nonce
        new_block.timestamp = datetime.now()
        new_block.hash = new_block.calculate_hash()

    return new_block

genesis_block = Block(0, datetime.now(), "Genesis Block", "0")
new_block = next_block(genesis_block, data="Transaction data")

print("New Block Details:")
print("Index:", new_block.index)
print("Timestamp:", new_block.timestamp)
print("Data:", new_block.data)
print("Previous Hash:", new_block.previous_hash)
print("Nonce:", new_block.nonce)
print("Hash:", new_block.hash)


New Block Details:
Index: 1
Timestamp: 2023-10-28 00:39:41.082767
Data: Transaction data
Previous Hash: e9c1b3f44b4b2084b3389adac110851cb5d336645a7a69e9ed203a077ca3eb02
Nonce: 5061
Hash: 0000d2ef92843cbc8ccfe8cb0dfcfb7062a1593a640f336bf3cf1b94cfecbec0


For the final part of this assignment, you need to create 20 blocks and link them together. Keep it
in mind that the mining difficulty can be different for these blocks.

In [7]:
import random

class Block:
    def __init__(self, index, timestamp, data, previous_hash, nonce, difficulty):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.difficulty = difficulty
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_contents = (
            str(self.index)
            + str(self.timestamp)
            + str(self.data)
            + str(self.previous_hash)
            + str(self.nonce)
        )
        return hashlib.sha256(block_contents.encode()).hexdigest()

def next_block(prev_block, data=''):
    index = prev_block.index + 1
    timestamp = datetime.now()
    previous_hash = prev_block.hash
    difficulty = random.randint(2, 3)  # Random difficulty between 2 and 6
    #difficulty = 2

    nonce = 0
    new_block = Block(index, timestamp, data, previous_hash, nonce, difficulty)
    while not new_block.hash.startswith('0' * new_block.difficulty):
        nonce += 1
        new_block.nonce = nonce
        new_block.timestamp = datetime.now()
        new_block.hash = new_block.calculate_hash()
    return new_block

# Create Genesis Block
genesis_block = Block(0, datetime.now(), "Genesis Block", "0", 0, 4)  # Initial difficulty

# Create 20 blocks and link them
chain = [genesis_block]
for i in range(1, 20):
    new_block = next_block(chain[-1], f"Transaction Data {i}")
    chain.append(new_block)

# Print details of the chain
for block in chain:
    print("Block Details:")
    print("Index:", block.index)
    print("Timestamp:", block.timestamp)
    print("Data:", block.data) 
    print("Previous Hash:", block.previous_hash)
    print("Nonce:", block.nonce)
    print("Difficulty:", block.difficulty)
    print("Hash:", block.hash)
    print("------------------------------------")


Block Details:
Index: 0
Timestamp: 2023-10-28 17:10:59.464234
Data: Genesis Block
Previous Hash: 0
Nonce: 0
Difficulty: 4
Hash: 8dd4b4f1c17892294fa4bc69b173c678ff8f83c6ae30152d7b76665bedbbad34
------------------------------------
Block Details:
Index: 1
Timestamp: 2023-10-28 17:10:59.466384
Data: Transaction Data 1
Previous Hash: 8dd4b4f1c17892294fa4bc69b173c678ff8f83c6ae30152d7b76665bedbbad34
Nonce: 4
Difficulty: 2
Hash: 006f3b7eb54d6bc9e98fbf7261ceed879dd5a7c995cb5285b2d690f3113a565c
------------------------------------
Block Details:
Index: 2
Timestamp: 2023-10-28 17:10:59.469664
Data: Transaction Data 2
Previous Hash: 006f3b7eb54d6bc9e98fbf7261ceed879dd5a7c995cb5285b2d690f3113a565c
Nonce: 345
Difficulty: 2
Hash: 0074b5a95519651515d539669b3623180817294c07d82fc18a96b8635eeca829
------------------------------------
Block Details:
Index: 3
Timestamp: 2023-10-28 17:10:59.469664
Data: Transaction Data 3
Previous Hash: 0074b5a95519651515d539669b3623180817294c07d82fc18a96b8635eeca829
Nonce